In [ ]:
"""
Dissolve using GIS Software
"""

import datetime
from gasp.mng.genze import dissolve

IN_SHP  = '/home/osmtolulc/webvia/tst_tools/cmb_roads.shp'
OUT_SHP = '/home/osmtolulc/webvia/tst_tools/cmb_diss_arc.shp'
FIELD   = 'roads'
API     = 'saga'

time_a = datetime.datetime.now().replace(microsecond=0)
dissolve(IN_SHP, OUT_SHP, FIELD, api=API)
time_b = datetime.datetime.now().replace(microsecond=0)

print time_b - time_a

In [ ]:
"""
Dissolve using SGBD Software
"""
import os
import datetime
from gasp.sql.mng.genze import st_dissolve
from gasp.sql.db import create_db
from gasp.to.psql import shp_to_psql_tbl
from gasp.oss import fprop

IN_SHP  = '/home/osmtolulc/webvia/tst_tools/cmb_roads.shp'
OUT_SHP = '/home/osmtolulc/webvia/tst_tools/cmb_diss_sqlit.shp'
FIELD   = 'roads'
EPSG    = 3857
API     = 'sqlite'

if API == 'psql':
    conParam = {
        'HOST' : 'localhost', 'PORT' : '5432', 'USER' : 'postgres',
        'PASSWORD' : 'admin', 'TEMPLATE' : 'postgis_template'
    
    }
    DB = 'teste'

    conParam["DATABASE"] = create_db(conParam, DB)

    tbl = shp_to_psql(conParam, IN_SHP, srsEpsgCode=EPSG, api='shp2pgsql')

else:
    from gasp.to.shp import shp_to_shp
    from gasp.anls.exct import sel_by_attr
    
    tbl = fprop(IN_SHP, 'fn')
    conParam = shp_to_shp(
        IN_SHP, os.path.join(
            os.path.dirname(IN_SHP), tbl + '.sqlite'
        ), gisApi='ogr', supportForSpatialLite=True
    )

time_a = datetime.datetime.now().replace(microsecond=0)
diss_tbl = st_dissolve(
    conParam, tbl,
    "geom" if API == 'psql' else 'geometry',
    fprop(OUT_SHP, 'fn'), diss_cols=FIELD, api=API
)
time_b = datetime.datetime.now().replace(microsecond=0)

print time_b - time_a

if API == 'psql':
    from gasp.to.shp import psql_to_shp
    
    psql_to_shp(conParam, diss_tbl, OUT_SHP, api='pgsql2shp')

else:
    sel_by_attr(conParam, "SELECT * FROM {}".format(diss_tbl), OUT_SHP)